# Processing and classifying images

In [ ]:
from transformers import image_transforms
from transformers import pipeline
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

## Processing image data

In [ ]:
original_image = Image.open("images/fashion.jpeg")

# Create the numpy array
image_array = np.array(original_image)

imgplot = plt.imshow(image_array)
plt.show()

In [ ]:
# Crop the center of the image
cropped_image = image_transforms.center_crop(image=image_array, size=(200, 200))

imgplot = plt.imshow(cropped_image)
plt.show()

## Creating an image classifier

In [ ]:
cropped_pil_image = Image.fromarray(cropped_image)
print(cropped_pil_image)

In [ ]:
# Create the pipeline
image_classifier = pipeline(task="image-classification", 
            model="abhishek/autotrain_fashion_mnist_vit_base")

# Predict the class of the image
results = image_classifier(cropped_pil_image)

# Print the results
print(results[0]["label"])

## What about the original image?

In [ ]:
# Predict the class of the image
results = image_classifier(original_image)

# Print the results
print(results[0]["label"])